Cartesian MRI
=============
The MRI signal can be described by
$$y(t) = \int_{\Omega}f(\vec{r}) e^{-i2\pi \vec{k}(t)\cdot \vec{r}}d\vec{r} \qquad \mathrm{[1]}$$  

where $\vec{r}$ denotes spatial position, $t$ denotes time, $f(\vec{r})$ is the object and $\vec{k}$ corresponds to the MRI $k$-space trajectory.

In discrete form (assuming a dirac pixel basis), Eq. 1 can be represented as:

$$y_i = \sum_{j=1}^{n_j}x_j e^{-i2\pi \vec{k_i}\cdot \vec{x_j}} \qquad \mathrm{[2]}$$

Where $j$ is an index over pixels ($n_j = N_xN_yN_z$) of an unknown object, $x$, and $i$ is a temporal index over the $k$-space data samples.

In this case, if the $k$-space samples are on a regular grid, the data is simply the discrete Fourier transform of the object.  This can be represented as:
$$ y = F x  \qquad \mathrm{[3]} $$ where $F$ is a discrete fourier transform (DFT) matrix.  In practice, the DFT is rapidly evaluated via a Fast Fourier Transform (FFT) as opposed to explicitly forming a large DFT matrix.




Non-Cartesian MRI
=================
When the k-space samples are not on a regular grid a non-uniform fast Fourier transform can be used instead.

$$ y = E x  \qquad \mathrm{[4]}$$ where $E$ represents the Non-uniform FFT (NUFFT).

Going forward, $E$ will be used to represent the transform from image space to k-space for both the Cartesian or Non-Cartesian cases.


Parallel Imaging
================
In parallel imaging, multiple MRI receive coils are employed simultaneously.  Let the sensitivity of a given coil, $c$, be given as a function of spatial position, $C_c( \vec{r})$.  The net signal received is then a sum over the signals from all coils:

$$y_c(t) = \int_{\Omega}C_c(\vec{r})f(\vec{r}) e^{-i2\pi \vec{k}(t)\cdot \vec{r}}d\vec{r} \qquad \mathrm{[5]}$$

Similarly to before this can be written compactly for a discretely sampled data in matrix-vector format.  Where the data vector, $y$ is now composed of a stack of the data from all individual coils.  Let the coils be indexed by the variable, $n=1,...,N$.


$$
    \underbrace{
        \begin{bmatrix} \vec{y}_1 \\ \vec{y}_2 \\ \vdots \\ \vec{y}_N
        \end{bmatrix}
    }_{\vec{y}}
    =
    \underbrace{
        \begin{bmatrix} EC_1 \\ EC_2 \\ \vdots \\ EC_N
        \end{bmatrix}
    }_{A}
    \vec{x} \qquad \mathrm{[6]}
$$

Each $C_n$ here is a diagonal matrix, where the diagonal corresponds to the coil sensitivity map, so that $C_n \vec{x}$ is the image modulated by the coil map for coil, $n$.

The Cartesian parallel imaging operator, $A$, of Eq. 6 is implemented in **`mrrt.mri.operators.MRI_Operator`**.  A boolean sample mask is set via the attribute `sample_mask`.


ESPIRIT (Soft Sense)
===========
The ESPIRIT technique can be used to estimate multiple sets of sensitivity maps from a fully sampled calibration region in k-space. Most often only 1 or two maps are estimated. The first set of maps correspond to coil sensitivities as used in a traditional SENSE reconstruction. Any additional maps account for componenets of the data due to motion, etc. that are not accounted for by the standard SENSE signal model. With this approach, multiple images, $\vec{x}^j$ for $j = 1, ..., M$ are produced.  Each of these corresponds to one of the $M$ different sets of sensitivity maps.

$$
    \underbrace{
        \begin{bmatrix} \vec{y}_1 \\ \vec{y}_2 \\ \vdots \\ \vec{y}_N
        \end{bmatrix}
    }_{\vec{y}}
    =\sum_{j=0}^M
    \underbrace{
        \begin{bmatrix} EC_1^j \\ EC_2^j \\ \vdots \\ EC_N^j
        \end{bmatrix}
    }_{A}
    \vec{x}^j \qquad \mathrm{[7]}
$$




Incorporating A Partial Sampling Mask
=====================================
For Cartesian MRI when only a subset of the samples are acquired, the above can be modified slightly as

$$
    \underbrace{
    \begin{bmatrix} 
        \vec{y}_1 \\ \vec{y}_2 \\ \vdots \\ \vec{y}_N
    \end{bmatrix}
    }_{\vec{y}}
    = 
    \underbrace{
      \begin{bmatrix}
         PFC_1 \\ PFC_2 \\ \vdots \\ PFC_N
      \end{bmatrix}
    }_{A}
    \vec{x} \qquad \mathrm{[8]}
$$

where P is a masking operation, that keeps only the k-space samples corresponding to the sampling mask.


The Cartesian parallel imaging operator, $A$, of Eq. 8 is implemented in **`mrrt.mri.operators.MRI_Cartesian`**.  A boolean sample mask is set via the attribute `sample_mask`.

Incorporating Off-Resonance
===========================

The presence of local field inhomogeneities can be modeled by extending the MRI signal equation to include the off-resonance field, $\omega(\vec{x})$.  In short, this term relfects the fact that the off-resonance induced phase at a given spatial location is the product of the time since RF excitation and the value of the off-resonance field at that position)
$$y = \int_{\Omega}f(\vec{r}) e^{-i\omega(\vec{r})t} e^{-i2\pi \vec{k}(t)\cdot \vec{r}}d\vec{r} \qquad \mathrm{[9]}$$

The discrete form for Eq. 9 is: 

$$y_i = \sum_{j=1}^{n_j}f_j e^{-i\omega_j t_i} e^{-i2\pi \vec{k_i}\cdot \vec{r_j}} \qquad \mathrm{[10]}$$

where $j$ is an index over discrete spatial positions (pixels or voxels) and $i$ is an index over temporal samples.  (Note: if $i\omega$ is replaced by $z = r + i\omega$, the $r$ term can be used to also take into account $\mathrm{R2}^*$ relaxation during the readout.

For Eq. 10, a simple Fourier relationship no longer holds, but we can make an approximation to the $e^{i\omega_j t_i}$ term that will still allow us to use FFTs to evaluate this term.  Here we use a least-squares time-segementation approach with min-max optimal interpolators [Sutton2004].  An implementation is available as mode ``hist,time,unif`` in the method ``mri_exp_approx`` of Jeff Fessler's Image Reconstruction Toolbox [REF].  This approach approximates the exponential term by temporal interpolation between exponentials corresponding to a discrete set of time points $\{t_{l}:l=1,...,L\}$.

$$e^{i\omega_j t_i} \approx \sum_{l=1}^{L} b_{l_{i}} e^{-i\omega_j t_{l}} \qquad \mathrm{[11]}$$


$$y = A x$$ where
$$ A \approx \sum_{l=1}^{L} D_{b_{l}} E D_{c_{l}} $$

Operator Incorporating Multiple Coils & Off-Resonance
=====================================================

$$
    \underbrace{
    \begin{bmatrix}
        \vec{y}_1 \\ \vec{y}_2 \\ \vdots \\ \vec{y}_{N}
    \end{bmatrix}
    }_{\vec{y}}
    =
    \underbrace{
    \begin{bmatrix}
        \sum_{l=1}^{L} D_{b_{l}} EC_1 D_{c_{l}} \\
        \sum_{l=1}^{L} D_{b_{l}} EC_2 D_{c_{l}} \\ \vdots \\
        \sum_{l=1}^{L} D_{b_{l}} EC_{N} D_{c_{l}}
    \end{bmatrix}
    }_{A}
    \vec{x} \qquad \mathrm{[12]}
$$

The Non-Cartesian parallel imaging operator, A, above is also implemented in **`pyir.operators.MRI_Operator`** when a field map or relaxation map is provided.

Simultaneous Reconstruction of Multiple Timepoints
==================================================
Suppose an acquisition involves the acquisition of mulitple images (volumes), each of which may have its own distinct k-space sampling pattern.  Some image reconstruction approaches involve joint reconstruction of the separate images.  Let the images, $x_j$ be indexed by $j=1,...,J$.  Let $A_j$ be the system matrix such as the one above for the $j$th sampling pattern.  Then the full system for all sampling patterns can be created by making a block diagonal system matrix as follows.

$$
    \underbrace{
        \begin{bmatrix}
            \vec{y}_{1} \\ \vec{y}_{2} \\ \vdots \\ \vec{y}_{J}
        \end{bmatrix}
    }_{\vec{y}}
    =
    \underbrace{
        \begin{bmatrix}
            A_1 & 0 & \cdots & 0 \\
            0 & A_2 & \cdots & 0 \\
            \vdots & \vdots & \ddots & \vdots \\
            0 & 0 & \cdots & A_{J}
         \end{bmatrix}
    }_{\mathbf{A}}
    \underbrace{
        \begin{bmatrix}
            \vec{x}_{1} \\ \vec{x}_{2} \\ \vdots \\ \vec{x}_{J}
        \end{bmatrix}
    }_{\vec{x}} \qquad \mathrm{[Eq: MultiFrame, 1]}
$$

where here, $y_j$ are the k-space data (across all coils) for each segment and $x_j$ are the image volumes for each segment.  The $A_i$ are the encoding operators for each segment.  In some cases these may be identical (i.e. if the same sampling pattern was used for all segments), but in general may involve different k-space sampling patterns.  

This sort of block diagonal operator can be built using **`mrrt.operators.BlockDiagLinOp`**.

Incorporating within-volume motion
==================================

Let $W_j$ represent a warping of segment $j$ to a common reference position (e.g. the central segment).  Depending on the type of motion, $W_j$ could be either a rigid, linear affine or non-linear transformation.  Then an operator representing k-space segmented into motion bins, indexed by $b=1,...,B$ can be given as:

For Cartesian MRI when only a subset of the samples are acquired, the above can be modified slightly as

$$
\underbrace{
    \begin{bmatrix}
        \vec{y}_1 \\ \vec{y}_2 \\ \vdots \\ \vec{y}_B
    \end{bmatrix}
}_{\vec{y}}
=
\underbrace{
    \begin{bmatrix}
        A_{1}W_{1} \\ A_{2}W_{2} \\ \vdots \\ A_{B}W_{B}
    \end{bmatrix}
}_{\mathbf{A}}
\vec{x} \qquad \mathrm{[Eq: Motion, 1]}
$$

So a single image undergoes $j$ different warps prior to multiplication by the sensitivity maps and transformation to k-space.  Where the k-space mask (or trajectory subset) used in operator $A_b$, corresponds to the k-space data corresponding to data collected during spatial position, $b$.  In the case of Fieldmap-corrected Non-Cartesian MRI each $A_b$ could also have its own unique fieldmap operators (e.g. if the segments correspond to different respiratory positions, it may be optimal to have a separate fieldmap for each).

Two operators such as $A_1$ and $W_1$ can be chained via **`mrrt.operators.CompositeLinOp`**

A vertically stacked operator, such as, $\mathbf{A}$ can be created via:  **`mrrt.operators.BlockColumnLinOp`**

Multiple volumes, each with sub-volume motion
=============================================
It is possible for each of the $A_0$ in *Eq: MultiFrame, 1* could correspond to a motion-segmented operator, $\mathbf{A}$ as defined in *Eq: Motion, 1*